In [1]:
import glob 
import os
import pandas as pd
from scipy.signal import find_peaks
from datetime import datetime
from datetime import timedelta

from soundbay.utils.metadata_processing import non_overlap_df

In [2]:
# prediction script: 
# for file in /opt/dlami/nvme/danielle/100_samples/*.wav ; do python inference.py -cn runs/inference_single_audio data.test_dataset.file_path="$file" experiment.checkpoint.path=../checkpoints/v5q3lg3h/best.pth data.data_sample_rate=96000 +data.sample_rate=24000 data.test_dataset.overlap=0.75 experiment.save_raven=yes; done

In [3]:
files_path = '/home/ubuntu/soundbay/outputs/*.csv'

In [9]:
samples = [x.split('.wav')[0].split('/')[-1] for x in glob.glob('/opt/dlami/nvme/danielle/audio/*.wav')]
models = ['v5q3lg3h', '9aj7fdoz']

In [10]:
def get_files(model, files):
    pred_dict = {
    model: [[
        x for x in glob.glob(files_path) if (x.split('-')[-2]==model) and (f in x)
               ][0] for f in files]
    for model in models 
    }
    
    return pred_dict[model]

In [14]:
model = 'v5q3lg3h'
raven_files = sorted(get_files(model, samples))
huwh_raven_files = sorted(get_files('9aj7fdoz', samples))

In [12]:
def big_df_from_outputs(files_path, first_class='Upsweeps', last_class='Squeaks', th=0.5, begin_time=3):
    df= pd.concat([
        pd.read_csv(f)
        .assign(**{
        'prob': lambda x: x.loc[:, first_class:last_class].max(axis=1).round(3),
        'Begin File': lambda x: f.split('-')[-1].split('.csv')[0] + '.wav', 
        'file': i, 
    }).pipe(lambda df: df[df['begin_time'] >= begin_time])
    for i, f in enumerate(files_path)
    ]).sort_values(['Begin File', 'begin_time']).reset_index(drop=True)

    return df

In [13]:
df = big_df_from_outputs(raven_files)

In [17]:
huwh =  big_df_from_outputs(huwh_raven_files, first_class='HUWH', last_class='HUWH').query('prob > 0.5')

In [19]:
heights = {
    'Upsweeps': 0.5, 
    'Downsweeps': 0.5, 
    'Tones': 0.5, 
    'Squeaks': 0.5
}

In [20]:
import numpy as np

indices = set()
for c in heights:  
    indices = indices | set(find_peaks(df[c], height=heights[c])[0])

peaks = df.iloc[list(indices)].sort_values(['file', 'begin_time']).reset_index(drop=True)

In [21]:
def merge_upsweep_downsweep(df): 
    df = df.sort_values('begin_time').reset_index(names='current_index')
    
    upsweep = df[
        (df.Downsweeps.shift(-1) >=  heights['Downsweeps'])
        & (df.Upsweeps >=  heights['Upsweeps']) 
        & (df.begin_time.shift(-1) <= df.end_time)]
    
    indices = []
    
    for i, call in upsweep.iterrows(): 
        idx = i + 1 
        end_time = call.end_time
        while (
            (idx < len(df)) and
            (df.iloc[idx]['Downsweeps'] >= heights['Downsweeps']) & 
            (df.iloc[idx]['begin_time'] <= end_time)
        ): 
            if idx - i <= 1: 
                df.loc[i, 'end_time'] = df.iloc[idx]['end_time']
            indices.append(idx)
            idx += 1

    return df.drop(indices).set_index('current_index')

In [22]:
peaks = peaks.groupby('filename').apply(merge_upsweep_downsweep).reset_index(drop=True)

/tmp/ipykernel_2987/1559189936.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  peaks = peaks.groupby('filename').apply(merge_upsweep_downsweep).reset_index(drop=True)


In [23]:
txt_columns = [
    'Selection',
    'View', 
    'Channel', 
    'Begin Time (s)', 
    'End Time (s)', 
    'Begin File', 
    'End File', 
    'Low Freq (Hz)',
    'High Freq (Hz)',
    'Annotation', 
]

In [24]:
save_df = (
    peaks
    .groupby('file')
    .apply(non_overlap_df, overlap_pct_th=0.75)
    .sort_values(['Begin File', 'begin_time'])
    .reset_index(drop=True)
    .assign(**{
        'end_time': lambda x: x.end_time.round(3), 
        'begin_time': lambda x: x.begin_time.round(3), 
        'End File': lambda x: x['Begin File'],
        'Begin Time (s)': lambda x: x['begin_time'] + (x['file']*840) , 
        'End Time (s)': lambda x: x['end_time']+ (x['file']*840),
        'Selection': lambda x: x.index + 1, 
        'View': 1, 
        'Channel': 0,
        'Low Freq (Hz)': 0, 
        'High Freq (Hz)': 48000, 
        'Annotation': lambda x: x.prob.astype(str) 
    })
)

/tmp/ipykernel_2987/1118464948.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  peaks


In [27]:
(
    huwh
    .groupby('Begin File')
    .apply(lambda g: sum(g.end_time - g.begin_time))
)

/tmp/ipykernel_2987/1909239496.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: sum(g.end_time - g.begin_time))


Begin File
5756.210617002958.wav    508.0
5756.210718162958.wav    112.0
5756.210720222958.wav     52.0
5756.211114132958.wav      4.0
5756.220415103455.wav     12.0
dtype: float64

In [32]:
meta_data = pd.concat([
    save_df
    .assign(duration=lambda df: df.end_time - df.begin_time)
    .groupby('Begin File')
    .agg(
        number_of_calls=pd.NamedAgg(column="begin_time", aggfunc=lambda g: len(g)),
        first_call_begin_sec=pd.NamedAgg(column="begin_time", aggfunc= lambda g: g.iloc[0]), 
        last_call_end_sec=pd.NamedAgg(column="end_time", aggfunc= lambda g: g.iloc[-1]), 
        call_duration=pd.NamedAgg(column="duration", aggfunc="sum"), 
        noise_duration=pd.NamedAgg(column="duration", aggfunc=lambda g: 840 - g.sum()),
        date=pd.NamedAgg(column="End File", aggfunc=lambda g: datetime.strptime(g.iloc[0].split('.')[1][:6], "%y%m%d").strftime('%Y-%m-%d')),
        time=pd.NamedAgg(column="End File", aggfunc=lambda g: datetime.strptime(g.iloc[0].split('.')[1][6:13], "%H%M%S").strftime('%H-%M-%S')),
        end_time=pd.NamedAgg(column="End File", aggfunc=lambda g: (datetime.strptime(g.iloc[0].split('.')[1][6:13], "%H%M%S")+ timedelta(minutes=14)).strftime('%H-%M-%S')),
    ),
    huwh
    .groupby('Begin File')
    .apply(lambda g: sum(g.end_time - g.begin_time))
    .rename('humpback_duration')
], axis=1).assign(
    humpback_duration= lambda df: df.humpback_duration.fillna(0), 
    is_humpback= lambda df: df.humpback_duration > df.call_duration)

/tmp/ipykernel_2987/3989475249.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: sum(g.end_time - g.begin_time))


In [33]:
meta_data

,number_of_calls,first_call_begin_sec,last_call_end_sec,call_duration,noise_duration,date,time,end_time,humpback_duration,is_humpback
Begin File,,,,,,,,,,
5756.210428082958.wav,1,313.125,314.625,1.500,838.500,2021-04-28,08-29-58,08-43-58,0.0,False
5756.210510072958.wav,1,35.250,36.750,1.500,838.500,2021-05-10,07-29-58,07-43-58,0.0,False
5756.210617002958.wav,80,50.625,835.125,128.250,711.750,2021-06-17,00-29-58,00-43-58,508.0,True
5756.210628182958.wav,2,459.000,819.750,3.000,837.000,2021-06-28,18-29-58,18-43-58,0.0,False
5756.210718162958.wav,1,76.500,78.000,1.500,838.500,2021-07-18,16-29-58,16-43-58,112.0,True
5756.210720222958.wav,1,498.750,500.250,1.500,838.500,2021-07-20,22-29-58,22-43-58,52.0,True
5756.210724052958.wav,1,148.875,150.375,1.500,838.500,2021-07-24,05-29-58,05-43-58,0.0,False
5756.210920132958.wav,4,406.125,830.250,6.000,834.000,2021-09-20,13-29-58,13-43-58,0.0,False
5756.211114132958.wav,2,105.375,112.875,3.000,837.000,2021-11-14,13-29-58,13-43-58,4.0,True


In [34]:
os.makedirs(f'{model}_predictions', exist_ok=True)
os.makedirs(f'{model}_predictions/predictions_per_file', exist_ok=True)

In [35]:
meta_data.to_csv(f'{model}_predictions/metadata.csv')

In [36]:
(
    pd.Series(
        list(set(samples) - set(save_df['Begin File'].str.split('.wav').apply(lambda x: x[0]))),
        name='file'
    ).apply(lambda x: x+'.wav').to_csv(f'{model}_predictions/noise_files.csv')
)

In [37]:
pd.Series(save_df['Begin File'].unique(), name='file').to_csv(f'{model}_predictions/call_files.csv')

In [38]:
save_df.to_csv(f'{model}_predictions/{model}.txt', sep='\t')

In [39]:
(
    save_df
    [txt_columns]
    .to_csv(f'{model}_predictions/predictions_all_files.txt', sep='\t')
)

In [20]:
(
    save_df
    .groupby('Begin File')
    .apply(lambda g: 
           g
           .assign(**{'Begin Time (s)': lambda x: x['begin_time'], 'End Time (s)': lambda x: x['end_time']})
           [txt_columns]
           .to_csv(f'{model}_predictions/predictions_per_file/predictions_{g.name.split(".wav")[0]}.txt', sep='\t')
          )
)

/tmp/ipykernel_6836/2564409062.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g:


""
